### Project: Summarization

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## A) Basic Prompt

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [3]:
text = """
Mojo is a new programming language that bridges the gap between research and production by combining the best of Python syntax with systems \
programming and metaprogramming. With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
Mojo is designed to solve a variety of AI development challenges that no other language can, because Mojo is the first programming language \
built from the ground-up with MLIR (a compiler infrastructure that’s ideal for heterogeneous hardware, from CPUs and GPUs, to various AI ASICs). \
We also designed Mojo as a superset of the Python because we love Python and its community, but we couldn’t realistically enhance Python to do all \
the things we wanted.
What we wanted was an innovative and scalable programming model that could target accelerators and other heterogeneous systems that are pervasive \
in the AI field. This meant a programming language with powerful compile-time metaprogramming, integration of adaptive compilation techniques, caching \
throughout the compilation flow, and other features that are not supported by existing languages.
And although accelerators are important, one of the most prevalent and sometimes overlooked “accelerators” is the host CPU. \
Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration units, but they also serve as the “fallback” \
for operations that specialized accelerators don’t handle, such as data loading, pre- and post-processing, and integrations with foreign systems. \
So it was clear that we couldn’t lift AI with just an “accelerator language” that worked with only specific processors.
"""

messages = [
    SystemMessage(content=' You are an expert copywriter with expertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT:{text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [4]:
llm.get_num_tokens(text)

311

In [5]:
summary_output = llm(messages)

In [6]:
print(summary_output.content)

Mojo is a new programming language that combines Python syntax with systems programming and metaprogramming. It aims to bridge the gap between research and production by allowing developers to write portable code that is faster than C and seamlessly integrates with the Python ecosystem. Mojo is designed to solve AI development challenges by utilizing MLIR, a compiler infrastructure ideal for heterogeneous hardware. It offers powerful compile-time metaprogramming, adaptive compilation techniques, caching, and other features not found in existing languages. Additionally, Mojo recognizes the importance of CPUs as accelerators and provides support for operations that specialized accelerators may not handle.


### Summarising Using Prompt Templates

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [8]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to  {language}.
'''

promp = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [9]:
llm.get_num_tokens(promp.format(text=text, language='English'))

333

In [10]:
chain = LLMChain(llm=llm, prompt=promp)
summary = chain.run({'text' :text, 'language' : 'english' })

In [11]:
print(summary)

Mojo is a new programming language that combines the best features of Python with systems programming and metaprogramming. It is designed to solve AI development challenges and is built with MLIR, making it ideal for heterogeneous hardware. Mojo is a superset of Python and offers powerful compile-time metaprogramming and other features not found in existing languages. It can target accelerators and other heterogeneous systems, including the host CPU, which is often overlooked but plays a crucial role in AI operations.


### Summarizing Using SuffDocumentChain

In [12]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI 
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [13]:
with open('files/sj.txt') as f:
    text = f.read()
    #text
docs = [Document(page_content=text)]  # you can split document if you want
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [14]:
template='''Write the concise summary of the following text.
TEXT: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [15]:
chain = load_summarize_chain(
    llm,
    chain_type = 'stuff',
    prompt=prompt,
    verbose = False
)
output_summary = chain.run(docs)

In [16]:
print(output_summary)

Steve Jobs, CEO of Apple Computer and Pixar Animation Studios, delivered a commencement address in 2005 where he shared his personal story about facing death and emphasized the importance of following one's passion and intuition. He encouraged the graduates to not waste their limited time living someone else's life and to have the courage to pursue their own dreams. He concluded with the message to "Stay Hungry. Stay Foolish."


### Summarizing Large Documents Using map_reduce

In [10]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI 
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [11]:
with open('files/sj.txt') as f:
    text = f.read()
    #text
docs = [Document(page_content=text)]  # you can split document if you want
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [19]:
llm.get_num_tokens(text)

1204

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

In [21]:
len(chunks)

2

In [22]:
chain =  load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose = False
)
output_summary = chain.run(chunks)

In [23]:
print(output_summary)

Steve Jobs, in his 2005 commencement address, discussed the significance of living each day to the fullest and following one's passion. He revealed his own battle with pancreatic cancer and emphasized the desire to make the most of life. Jobs also encouraged the audience to embrace change, live their own lives, and stay hungry and foolish.


In [24]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [25]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

### MapReduce with Custom Prompts

In [26]:
map_prompt='''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''

map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [27]:
combine_prompt='''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])

In [28]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    verbose=False #set to true to see intermediate steps
)
output = summary_chain.run(chunks)

In [29]:
print(output)

Steve Jobs delivered a commencement address in 2005 where he discussed the importance of finding one's passion, living each day to the fullest, and embracing mortality. He shared his personal experience with pancreatic cancer and emphasized the need to prioritize what truly matters. The speech encourages individuals to live authentically and follow their own desires, not being influenced by others. Jobs also references The Whole Earth Catalog and its message of staying hungry and foolish as a wish for the graduating audience.


### Summarizing Using Refine Combined Chain

In [ ]:
#pip install unstructured -q

In [ ]:
#pip install pdfminer.six

In [ ]:
#pip install  pikepdf 

In [18]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI 
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyPDFLoader

In [19]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [20]:
#loader = UnstructuredPDFLoader('files/attention_is_all_you_need.pdf', mode="elements", strategy="fast",)
loader = PyPDFLoader('files/attention_is_all_you_need.pdf')
data = loader.load()

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.split_documents(data)

In [25]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [24]:
len(chunks)

15

In [26]:
def print_embedding_cost(texts):
    import tiktoken
    enc= tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens/1000*0.0004:.6f}')

print_embedding_cost(chunks)

Total Tokens: 10162
Embedding Cost in USD: 0.004065


In [27]:
chain =  load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose = False
)
output_summary = chain.run(chunks)

In [28]:
print(output_summary)

A majority of American governments have passed new laws since 2009 to make the registration or voting process more difficult. This is illustrated in Figure 3, which shows the attention mechanism in layer 5 of 6. The attention heads in the encoder self-attention focus on the distant dependency of the verb 'making', completing the phrase 'making...more difficult'. The new context provided does not add any relevant information to the original summary. Therefore, the original summary remains unchanged.


### Refine with Custom Prompts

In [29]:
prompt_template = '''
Write a concise and short summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:'''
initial_prompt = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = '''
    Your job is to produce a final summary.
    I have provided an existing summary up to a certain point: {existing_answer}.
    Please refine the existing summary with some mor context below.
    ----------
    {text}
    ----------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE
'''
refine_prompt = PromptTemplate(
    template=refine_template, 
    input_variables=['existing_answer', 'text']
)

In [30]:
chain =  load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps = False
)
output_summary = chain.run(chunks)

In [31]:
print(output_summary)

The Transformer is a groundbreaking network architecture that relies solely on attention mechanisms for sequence transduction tasks. It addresses the limitations of recurrent neural networks and achieves state-of-the-art results in machine translation and other tasks. The model consists of stacked self-attention and fully connected layers, making it the first transduction model to rely solely on self-attention. The paper introduces Scaled Dot-Product Attention and Multi-Head Attention, which further enhance the performance of the Transformer. The potential benefits of restricting self-attention to consider only a neighborhood of the input sequence and the possibility of self-attention yielding more interpretable models are discussed. The paper provides insights into the training regime for the models, including details about the training data, batching, hardware, schedule, optimizer, and regularization techniques. The Transformer achieves better BLEU scores than previous state-of-the-a

### Summarizing Using LangChain Agents

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [33]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [34]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
wikipedia = WikipediaAPIWrapper()

In [35]:
tools = [
    Tool(
        name='Wikipedia',
        func=wikipedia.run,
        description='Useful for when you nee to get information from wikipedia about a topic'
    )
]

In [37]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [38]:
output = agent_executor.run('Can you please provide a short summary of George Washington?')



> Entering new AgentExecutor chain...
I should look up George Washington on Wikipedia to get a short summary of his life and accomplishments.
Action: Wikipedia
Action Input: George Washington
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, and statesman who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in June 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted and ratified the Constitution of the United States and established the American federal government. Washington has thus been called the "Father of his Country".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia. He subsequently received milita

In [39]:
print(output)

George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, and statesman who served as the first president of the United States from 1789 to 1797. He played a crucial role in the American Revolutionary War, led the drafting and ratification of the Constitution, and set important precedents for the presidency. He is widely regarded as one of the greatest presidents in American history.
